In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("books").config('spark.master', 'local[*]').getOrCreate()

24/07/23 16:29:04 WARN Utils: Your hostname, themjdex-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/07/23 16:29:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/23 16:29:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
books = spark.read.csv('data/books.csv', header=True, inferSchema=True)
authors = spark.read.csv('data/authors.csv', header=True, inferSchema=True)

In [4]:
authors.printSchema()

root
 |-- author_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- country: string (nullable = true)



In [5]:
books.printSchema()

root
 |-- book_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: double (nullable = true)
 |-- publish_date: date (nullable = true)



In [6]:
# Объедините таблицы books и authors по author_id.
df = books.join(authors, 'author_id', 'inner').withColumnRenamed('author_id', 'author_id')
df.show()

+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|author_id|book_id|  title|      genre|price|publish_date|     name|birth_date|  country|
+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|        2|      1| Book_1|    Mystery|73.57|  1980-12-31| Author_2|1965-12-31|   Canada|
|        1|      2| Book_2|Non-Fiction| 41.1|  1982-12-31| Author_1|1960-12-31|    India|
|       10|      3| Book_3|    Fiction|10.63|  1984-12-31|Author_10|2005-12-31|    India|
|        9|      4| Book_4|Non-Fiction|46.31|  1986-12-31| Author_9|2000-12-31|Australia|
|        7|      5| Book_5|    Science|31.13|  1988-12-31| Author_7|1990-12-31|      USA|
|        4|      6| Book_6|Non-Fiction| 83.7|  1990-12-31| Author_4|1975-12-31|       UK|
|        6|      7| Book_7|Non-Fiction|40.36|  1992-12-31| Author_6|1985-12-31|      USA|
|        2|      8| Book_8|Non-Fiction|84.48|  1994-12-31| Author_2|1965-12-31|   Canada|
|        7

In [7]:
# Найдите топ-5 авторов, книги которых принесли наибольшую выручку.
top_authors = df.groupBy('author_id').sum('price'). \
                sort(F.desc('sum(price)')).limit(5)

top_authors.join(df, 'author_id', 'inner').select('author_id', 'name', 'sum(price)').distinct().sort(F.desc('sum(price)')).show()

+---------+--------+----------+
|author_id|    name|sum(price)|
+---------+--------+----------+
|        2|Author_2|    231.97|
|        7|Author_7|    132.66|
|        1|Author_1|    111.86|
|        8|Author_8|    107.16|
|        5|Author_5|     88.83|
+---------+--------+----------+



In [8]:
# Найдите количество книг в каждом жанре.
df.groupBy('genre').count().show()

+-----------+-----+
|      genre|count|
+-----------+-----+
|    Science|    3|
|    Fiction|    3|
|    Fantasy|    3|
|    Mystery|    2|
|Non-Fiction|    9|
+-----------+-----+



In [9]:
# Подсчитайте среднюю цену книг по каждому автору.
avg_authors = df.groupBy('author_id').avg('price')

avg_authors.join(df, 'author_id', 'inner'). \
select('author_id', 'name', 'avg(price)'). \
distinct(). \
sort(F.desc('avg(price)')). \
show()

+---------+---------+-----------------+
|author_id|     name|       avg(price)|
+---------+---------+-----------------+
|        5| Author_5|            88.83|
|        4| Author_4|             83.7|
|        2| Author_2|          57.9925|
|        9| Author_9|            46.31|
|        7| Author_7|            44.22|
|        6| Author_6|           43.965|
|        1| Author_1|37.28666666666667|
|        8| Author_8|            35.72|
|       10|Author_10|           21.165|
+---------+---------+-----------------+



In [10]:
# Найдите книги, опубликованные после 2000 года, и отсортируйте их по цене
df.filter(F.year('publish_date') > '2000').sort(F.desc('price')).show()

+---------+-------+-------+-----------+-----+------------+--------+----------+---------+
|author_id|book_id|  title|      genre|price|publish_date|    name|birth_date|  country|
+---------+-------+-------+-----------+-----+------------+--------+----------+---------+
|        7|     20|Book_20|    Mystery|91.48|  2018-12-31|Author_7|1990-12-31|      USA|
|        5|     19|Book_19|    Science|88.83|  2016-12-31|Author_5|1980-12-31|      USA|
|        8|     15|Book_15|    Fantasy| 60.0|  2008-12-31|Author_8|1995-12-31|Australia|
|        6|     17|Book_17|    Fantasy|47.57|  2012-12-31|Author_6|1985-12-31|      USA|
|        1|     18|Book_18|Non-Fiction|43.92|  2014-12-31|Author_1|1960-12-31|    India|
|        2|     16|Book_16|    Fiction|36.22|  2010-12-31|Author_2|1965-12-31|   Canada|
|        8|     12|Book_12|Non-Fiction|31.02|  2002-12-31|Author_8|1995-12-31|Australia|
|        1|     14|Book_14|    Fiction|26.84|  2006-12-31|Author_1|1960-12-31|    India|
|        8|     13|Bo

In [11]:
spark.stop()